Streamlit app: http://localhost:8501
GitHub-repo: https://github.com/marensofiesteen/IND320-streamlit/tree/main

Purpose: 
Briefly document the data, preprocessing, and a simple EDA (exploratory data analasis) as the fundation for the Streamlit app.

App:
Interactive visualization can be found in ´main.py´

Data:
File: ´open-meteo-subset.csv´
Elhub: https://api.elhub.no/energy-data/v0/price-areas

AI-use:
I combined the use of ChatGPT, VS Code Copilot and collaboration with others. I used VS code copilot to better understand the code I was writing, and if that wasnt enough, I asked chatGPT for a more detailed explanation. Collaboration with my peers was mainly for inspiration and guidance.

Log:
I started by installing Docker Desktop and signed in. 
Notebook:
    I first established a connection to Spark and Cassandra. From Cassandra, I retrived production data stored in the keyspace elnub and then the table production_data. After loading the data, I exported it to MongoDB. This was done in batches to prevent memory issues with large datasets. Exporting was completed by inserting the data into a database and collection in MongoDB. When the processing was done, the Spark session was stopped to free system resources. 
Main.py:
    I built an interactive web application in Streamlit to display and analyze the data. For the Production (Elhub) page, I made a connection to MongoDB and visualized electricity production by price area and production group using pie charts and time-series line charts. To securely access MongoDB from Streamlit, I used st.secrets so that usernames and passwords are not stored openly in the code. Errors are handled using try/except, and the user is informed if the database connection fails or containt no data. 
Terminal:
    I used Docker to start Cassandra. This ensured that the database and tables were available for Spark.
    MongoDB was either accessed via MongoDB Altlas (cloud service) or started locally on the computer. After both services were running, the notebook and Streamlit application were able to connect and use them

## API
- Time encoded: Elhubs API and open files use standard ISO-8601 timestamps, commonly in UTC (ex. 2025-10-22/10:00:00z). Using UTC avoids ambiguous local times around DST. (See the Energy Data API and API portal)
- Transitions between summer and winter: If you convert UTC result to Europe/Oslo, DST creates 23-hour days in spring (missing local 02:00) and 25-hour days in autumn (duplicate local 02:00). Handle this on the client when displaying or aggregating by local time (do not expect the API to "fix" DST for you)
- Time period limitations: Each dataset/endpoint has its own maximum query window (you must oage or loop long ranges). Elhub also offers Downloads-CSV files with the last 3 years, updated daily-useful for quick chechs without auth.
- How this differs between datasets: Some datasets are open (downloads), while others require Maskinporten scopes (authorizatiob set per dataset/access level).

In [24]:
EXECUTE_API = True  # Avoid execute api when convert into html file
EXECUTE_localDB = True   # Avoid execute database when convert into html file
EXECUTE_remoteDB = False 

import json
import os
from pyjstat import pyjstat
import requests
from cassandra.cluster import Cluster

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, to_date, hour, lit, when, isnan, count

import pandas as pd
import pytz
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import math

Step 1: Load data with API and insert the data to Cassandra with Spark

In [25]:
## 1. Preparation in Cassandra

# Connect to the Cassandra cluster from Python
cluster = Cluster(['127.0.0.1'], port=9042)  # Replace with your Cassandra cluster address
session = cluster.connect()

# Set up new keyspace
if EXECUTE_localDB:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS elnub WITH REPLICATION = {
        'class': 'SimpleStrategy',
        'replication_factor': 1}
    """)

# Create a table, if not exists makes sure we do not overwrite existing tables
if EXECUTE_localDB:
    session.set_keyspace('elnub')
    session.execute("DROP TABLE IF EXISTS elnub.production_data;")
    session.execute("CREATE TABLE IF NOT EXISTS elnub.production_data (\
                    priceArea TEXT,\
                    productionGroup TEXT,\
                    startTime TIMESTAMP,\
                    quantityKwh DOUBLE,\
                    endTime TIMESTAMP,\
                    lastUpdatedTime TIMESTAMP,\
                    PRIMARY KEY ((priceArea, productionGroup), startTime)\
                    );")

In [26]:
## 2. Preparation with Spark

# Set environment variables for PySpark
os.environ ["JAVA_HOME"] = "/opt/homebrew/Cellar/openjdk@17/17.0.17/libexec/openjdk.jdk/Contents/Home"

os.environ["PYSPARK_PYTHON"] = "/Users/marenssteen/Documents/IND320/Streamlit/.venv/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/Users/marenssteen/Documents/IND320/Streamlit/.venv/bin/python"

os.environ["SPARK_LOCAL_IP"] = "127.0.0.1" # To avoid binding issues on Mac M1

In [27]:
# Create Spark session
# NB: It is necessary to adapt the info based on your setup
spark = SparkSession.builder.appName('SparkCassandraApp').\
    config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.5.1').\
    config('spark.cassandra.connection.host', '127.0.0.1').\
    config('spark.sql.extensions', 'com.datastax.spark.connector.CassandraSparkExtensions').\
    config('spark.sql.catalog.mycatalog', 'com.datastax.spark.connector.datasource.CassandraCatalog').\
    config('spark.cassandra.connection.port', '9042').\
    config("spark.driver.bindAddress", "127.0.0.1").\
    config("spark.driver.host", "127.0.0.1").getOrCreate()

print("✅ Spark session started successfully!")
print("Spark version:", spark.version)
print("Packages:", spark.sparkContext.getConf().get("spark.jars.packages"))

✅ Spark session started successfully!
Spark version: 3.5.1
Packages: com.datastax.spark:spark-cassandra-connector_2.12:3.5.1


In [28]:
def get_period_start_end(year, m):
    
    # Generate a whole month time interval corresponding to the Norwegian time zone for the specified year and month.
    
    norway_tz = pytz.timezone("Europe/Oslo")
    
    start_date = norway_tz.localize(datetime(year, m, 1))
    if m == 12:
        end_date = norway_tz.localize(datetime(year + 1, 1, 1)) - timedelta(hours=1)
    else:
        end_date = norway_tz.localize(datetime(year, m + 1, 1)) - timedelta(hours=1)
    return start_date, end_date


In [29]:
def load_parse_production_data(start_dt, end_dt):
    
    # Obtain production data for a defined time period from the Elhub API and parse the data into a list.
    
    url = "https://api.elhub.no/energy-data/v0/price-areas"
    params = {
        "dataset": "PRODUCTION_PER_GROUP_MBA_HOUR",
        "startDate": start_dt.isoformat(),
        "endDate": end_dt.isoformat()
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Failed to get data for {start_dt} - {end_dt}: status {response.status_code}")
        return []
    
    data_json = response.json()
    parsed_data = []
    for data in data_json['data']:
        for item in data['attributes']['productionPerGroupMbaHour']:
            parsed_data.append(item)
    return parsed_data

In [30]:
def write_to_cassandra_ved_spark(data_list, table="production_data", keyspace="elnub"):
    
    # Writes a list of electricity production data to a local Cassandra database with spark.
    
    if not data_list:
        return

    df = pd.DataFrame(data_list)
    # define the low-case columns
    df.columns = ['endtime','lastupdatedtime','pricearea','productiongroup','quantitykwh','starttime']

    # Use spark to insert data into cassandra
    spark.createDataFrame(df)\
        .write\
        .format("org.apache.spark.sql.cassandra")\
        .options(table=table, keyspace=keyspace)\
        .mode("append")\
        .save()

Loop every month

In [31]:
start_year = 2021
end_year = 2021

if EXECUTE_API:
    for m in range(1, 13):
        start_dt, end_dt = get_period_start_end(start_year, m)
        # Special handling for October DST change
        if m == 10:
            norway_tz = pytz.timezone("Europe/Oslo")
            
            # The API limits retrieval to one month at a time, and October data initially failed due to the Daylight Saving Time transition, which added an extra hour. 
            # I resolved this by splitting October into two parts.
            
            end_first_part = norway_tz.localize(datetime(start_year, 10, 30, 23))
            parts = [(start_dt, end_first_part),
                    (norway_tz.localize(datetime(start_year, 10, 31, 0)),
                    norway_tz.localize(datetime(start_year, 10, 31, 23)))]
        else:
            parts = [(start_dt, end_dt)]
        
        for start_part, end_part in parts:
            data_list = load_parse_production_data(start_part, end_part)
            write_to_cassandra_ved_spark(data_list)
        
        print(f"Month {m} finished.")

Month 1 finished.
Month 2 finished.
Month 3 finished.
Month 4 finished.
Month 5 finished.
Month 6 finished.
Month 7 finished.
Month 8 finished.
Month 9 finished.
Month 10 finished.
Month 11 finished.
Month 12 finished.


2: Extract data with spark and plot figures

In [32]:
# Load data from Cassandra to notebook
df_from_cassandra = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="production_data", keyspace="elnub") \
    .load()

df_from_cassandra.show(5)

+---------+---------------+-------------------+-------------------+-------------------+-----------+
|pricearea|productiongroup|          starttime|            endtime|    lastupdatedtime|quantitykwh|
+---------+---------------+-------------------+-------------------+-------------------+-----------+
|      NO2|          solar|2021-01-01 00:00:00|2021-01-01 01:00:00|2024-12-20 10:35:40|    876.556|
|      NO2|          solar|2021-01-01 01:00:00|2021-01-01 02:00:00|2024-12-20 10:35:40|    876.398|
|      NO2|          solar|2021-01-01 02:00:00|2021-01-01 03:00:00|2024-12-20 10:35:40|    876.545|
|      NO2|          solar|2021-01-01 03:00:00|2021-01-01 04:00:00|2024-12-20 10:35:40|    876.929|
|      NO2|          solar|2021-01-01 04:00:00|2021-01-01 05:00:00|2024-12-20 10:35:40|    877.317|
+---------+---------------+-------------------+-------------------+-------------------+-----------+
only showing top 5 rows



In [33]:
# Extract the required columns
df_spark = df_from_cassandra.select("pricearea", "productiongroup", "starttime", "quantitykwh")
df_spark.columns

['pricearea', 'productiongroup', 'starttime', 'quantitykwh']

In [34]:
# Explore the dataset
df_spark.describe().show()

+-------+---------+---------------+------------------+
|summary|pricearea|productiongroup|       quantitykwh|
+-------+---------+---------------+------------------+
|  count|   215033|         215033|            215033|
|   mean|     NULL|           NULL| 729742.5154550363|
| stddev|     NULL|           NULL|1549796.6064128485|
|    min|      NO1|          hydro|               0.0|
|    max|      NO5|           wind|         9715193.0|
+-------+---------+---------------+------------------+



In [35]:
df_spark.printSchema()

root
 |-- pricearea: string (nullable = false)
 |-- productiongroup: string (nullable = false)
 |-- starttime: timestamp (nullable = true)
 |-- quantitykwh: double (nullable = true)



In [36]:
total_rows = df_spark.count()
print(f"Total rows in Spark DataFrame: {total_rows}")

Total rows in Spark DataFrame: 215033


In [37]:
df_spark.groupBy("productiongroup", "pricearea")\
    .count()\
    .orderBy("productiongroup", "pricearea", ascending=True)\
    .show(50)

+---------------+---------+-----+
|productiongroup|pricearea|count|
+---------------+---------+-----+
|          hydro|      NO1| 8747|
|          hydro|      NO2| 8747|
|          hydro|      NO3| 8747|
|          hydro|      NO4| 8747|
|          hydro|      NO5| 8747|
|          other|      NO1| 8747|
|          other|      NO2| 8747|
|          other|      NO3| 8747|
|          other|      NO4| 8747|
|          other|      NO5| 8747|
|          solar|      NO1| 8747|
|          solar|      NO2| 8747|
|          solar|      NO3| 8747|
|          solar|      NO4| 8747|
|          solar|      NO5| 8747|
|        thermal|      NO1| 8747|
|        thermal|      NO2| 8747|
|        thermal|      NO3| 8747|
|        thermal|      NO4| 8747|
|        thermal|      NO5| 8747|
|           wind|      NO1| 8747|
|           wind|      NO2| 8747|
|           wind|      NO3| 8747|
|           wind|      NO4| 8747|
|           wind|      NO5| 5105|
+---------------+---------+-----+



In [38]:
df_clean = df_spark.dropna(subset=['starttime', 'quantitykwh', 'pricearea', 'productiongroup'])
df_clean = df_clean.dropDuplicates(['pricearea', 'productiongroup', 'starttime'])

df_clean.count()

215033

Pie chart: with plotly module to get a clearer pie chart (hence the small categories)

In [39]:
# Filter and aggregate data
df_sub1 = df_spark.filter(df_spark.pricearea == 'NO1') \
    .groupBy('productiongroup') \
    .agg(sum('quantitykwh') \
    .alias('total_kwh'))

df_sub1_pd = df_sub1.toPandas()

# Sort the values
df_sub1_pd.sort_values(by='total_kwh', ascending=False)

,productiongroup,total_kwh
1,hydro,1.833124e+10
3,wind,5.464368e+08
0,thermal,2.357448e+08
2,solar,1.438160e+07
4,other,5.255678e+04


In [40]:
# Use the pie function to plot

fig = px.pie(
    df_sub1_pd,
    values='total_kwh',
    names='productiongroup',
    title=f"Production Distribution by Group (NO1, 2021)",
)

# Modify the structure of the plot
fig.update_traces(
    textinfo='percent+label', # display the percent and label for each pie
    pull=[0.05]*len(df_sub1_pd), # seperate each pie slightly
    textfont_size=15, # define the size of text
)

# Define the layout of the whole plot, like the layout of the legend, title
fig.update_layout(
    width=1200,
    height=600,
    legend_title_text='Production Groups',
    legend=dict(
        font=dict(size=15)
    ),
    title=dict(
        font=dict(size=18)
    )
)
# show the plot
fig.show()

Line plot

In [41]:
from pyspark.sql.functions import col, month
# Filter data
df_sub2 = (
    df_spark
    .filter((col("pricearea") == "NO1") & (month(col("starttime")) == 1))
)
# Convert to Pandas for plotting
df_sub2_pd = df_sub2.toPandas()

In [42]:
# use the line function to plot and each line represent each production group
fig = px.line(
    df_sub2_pd,
    x="starttime",
    y="quantitykwh",
    color="productiongroup",
    title="Hourly Production Distribution (NO1, 2021 Jan)",
    labels={ # rename the lanels
        "starttime": "Days",
        "quantitykwh": "Production (kWh)",
        "productiongroup": "Production Group"
    } 
)
fig.show()

Step 3: Push to into MongoDB

In [43]:
# 0) Read from Cassandra
df = (spark.read
      .format("org.apache.spark.sql.cassandra")
      .options(table="production_data", keyspace="elnub")
      .load())

total = df.count()
print("Cassandra rows:", total)

# 1) Connect to MongoDB (local)
mongo_uri = "mongodb://localhost:27017"   # or use Atlas-URI if you want
client = MongoClient(mongo_uri)
coll = client["elhub_db"]["production_data"]

# Clear existing data, if any
coll.delete_many({})
print("Mongo cleared.")

# 2) Batch-size (tuning): 10_000 rows works well for 200k+ rows
BATCH = 10_000
num_batches = math.ceil(total / BATCH)
print(f"Exporting in ~{num_batches} batches of {BATCH} rows")

# 3) Export batches by collecting per-partition to smaller Pandas-chunks
# (Used toLocalIterator to streame without collecting everythint to the driver at ones)
buffer = []
count_written = 0

for row in df.toLocalIterator():
    # Convert one by one Spark-ros to python-dict
    d = row.asDict(recursive=True)

    # Mongo does not allow NaN/NaT → convert through pandas (fast and safe)
    tmp = pd.DataFrame([d]).where(pd.notnull(pd.DataFrame([d])), None)
    d_clean = tmp.iloc[0].to_dict()

    buffer.append(d_clean)

    if len(buffer) >= BATCH:
        coll.insert_many(buffer)
        count_written += len(buffer)
        print(f"Wrote {count_written}/{total}")
        buffer = []

# flush the last buffer
if buffer:
    coll.insert_many(buffer)
    count_written += len(buffer)
    print(f"Wrote {count_written}/{total}")

print("✅ Export to MongoDB complete.")
print("Mongo count now:", coll.count_documents({}))


Cassandra rows: 215033
Mongo cleared.
Exporting in ~22 batches of 10000 rows
Wrote 10000/215033
Wrote 20000/215033
Wrote 30000/215033
Wrote 40000/215033
Wrote 50000/215033
Wrote 60000/215033
Wrote 70000/215033
Wrote 80000/215033
Wrote 90000/215033
Wrote 100000/215033
Wrote 110000/215033
Wrote 120000/215033
Wrote 130000/215033
Wrote 140000/215033
Wrote 150000/215033
Wrote 160000/215033
Wrote 170000/215033
Wrote 180000/215033
Wrote 190000/215033
Wrote 200000/215033
Wrote 210000/215033
Wrote 215033/215033
✅ Export to MongoDB complete.
Mongo count now: 215033


In [44]:
spark.stop()